In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing data from http://web.mta.info/developers/turnstile.html from January to April 2021, a total of 13 weeks.
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week_num in week_nums: #for each week
        file_url = url.format(week_num) #format the url so it grabs that week
        dfs.append(pd.read_csv(file_url)) #append to our empty list
    return pd.concat(dfs) #combine files for each week
        
week_nums = [210123, 210130,210206,210213,210220,210227,210306,210313,210320,210327,210403, 210410, 210417]
turnstiles_df = get_data(week_nums)


In [3]:
#Exporting Data to .CSV 
turnstiles_df.to_csv('mta_data_all.csv', header = False, index = False)

In [4]:
turnstiles_df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,03:00:00,REGULAR,7518158,2561766
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,07:00:00,REGULAR,7518162,2561773
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,11:00:00,REGULAR,7518183,2561806
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,15:00:00,REGULAR,7518267,2561847
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,19:00:00,REGULAR,7518403,2561864
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,23:00:00,REGULAR,7518490,2561872
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/17/2021,03:00:00,REGULAR,7518496,2561874
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/17/2021,07:00:00,REGULAR,7518503,2561882
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/17/2021,11:00:00,REGULAR,7518525,2561908
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/17/2021,15:00:00,REGULAR,7518590,2561940


In [5]:
# Checking for null values
turnstiles_df.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2717672 entries, 0 to 209690
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count    Dtype 
---  ------                                                                --------------    ----- 
 0   C/A                                                                   2717672 non-null  object
 1   UNIT                                                                  2717672 non-null  object
 2   SCP                                                                   2717672 non-null  object
 3   STATION                                                               2717672 non-null  object
 4   LINENAME                                                              2717672 non-null  object
 5   DIVISION                                                              2717672 non-null  object
 6   DATE                                                                  2717672 non-n

### Loading Data into SQL

Using SQLAlchemy, data loaded into SQLite for querring.
 



In [6]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///mta.db")
all_tables = engine.table_names()
all_tables

['mta_data']

In [7]:
mta_data = pd.read_sql('SELECT * FROM mta_data;', engine)
mta_data

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,03:00:00,REGULAR,7518158,2561766
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,07:00:00,REGULAR,7518162,2561773
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,11:00:00,REGULAR,7518183,2561806
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,15:00:00,REGULAR,7518267,2561847
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,19:00:00,REGULAR,7518403,2561864
...,...,...,...,...,...,...,...,...,...,...,...
2717667,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,04/16/2021,05:00:00,REGULAR,5554,547
2717668,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,04/16/2021,07:15:02,REGULAR,5554,547
2717669,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,04/16/2021,13:00:00,REGULAR,5554,547
2717670,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,04/16/2021,17:00:00,REGULAR,5554,547


### Querying Data using SQL

In [8]:
#Each unique turnstile records 6 entries each day 
pd.read_sql("Select * from mta_data where CA = 'A002' and UNIT = 'R051' and SCP = '02-00-00' and STATION = '59 ST';", engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,03:00:00,REGULAR,7518158,2561766
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,07:00:00,REGULAR,7518162,2561773
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,11:00:00,REGULAR,7518183,2561806
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,15:00:00,REGULAR,7518267,2561847
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,19:00:00,REGULAR,7518403,2561864
...,...,...,...,...,...,...,...,...,...,...,...
540,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/16/2021,04:00:00,REGULAR,7557261,2580948
541,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/16/2021,08:00:00,REGULAR,7557273,2581011
542,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/16/2021,12:00:00,REGULAR,7557325,2581142
543,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/16/2021,16:00:00,REGULAR,7557494,2581182


In [9]:
#Looking for duplicates, by grouping unique features which should return count of 1 entry. 
#But looks like rows with Desc RECOVR AUD are duplicates

pd.read_sql('SELECT CA, UNIT, SCP, STATION, DATE, TIME, DESC, count(ENTRIES) from mta_data GROUP BY CA, UNIT, SCP, STATION, DATE, TIME ORDER BY count(ENTRIES) DESC;', engine).head(10)

,CA,UNIT,SCP,STATION,DATE,TIME,DESC,count(ENTRIES)
0,N071,R013,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
1,N071,R013,00-00-01,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
2,N071,R013,00-00-02,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
3,N071,R013,00-00-03,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
4,N071,R013,00-00-04,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
5,N071,R013,00-06-00,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,2
6,N100,R252,00-00-00,HIGH ST,03/06/2021,03:00:00,RECOVR AUD,2
7,N100,R252,00-00-00,HIGH ST,03/06/2021,07:00:00,RECOVR AUD,2
8,N100,R252,00-00-00,HIGH ST,03/06/2021,11:00:00,RECOVR AUD,2
9,N100,R252,00-00-00,HIGH ST,03/06/2021,19:00:00,RECOVR AUD,2


In [10]:
#Let's see how many RECOVR AUDs are present in the data
pd.read_sql("SELECT * from mta_data where DESC = 'RECOVR AUD';", engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A030,R083,01-00-00,23 ST,NRW,BMT,01/19/2021,15:00:00,RECOVR AUD,50372,54948
1,A030,R083,01-00-01,23 ST,NRW,BMT,01/19/2021,15:00:00,RECOVR AUD,7300598,2666467
2,A030,R083,01-00-02,23 ST,NRW,BMT,01/19/2021,15:00:00,RECOVR AUD,5490627,2539982
3,A030,R083,01-03-00,23 ST,NRW,BMT,01/19/2021,15:00:00,RECOVR AUD,6855259,3294214
4,A030,R083,01-03-01,23 ST,NRW,BMT,01/19/2021,15:00:00,RECOVR AUD,10457559,3214392
...,...,...,...,...,...,...,...,...,...,...,...
11514,R627,R063,00-03-00,SUTTER AV-RUTLD,3,IRT,04/12/2021,08:00:00,RECOVR AUD,1001626,1014721
11515,R627,R063,00-03-01,SUTTER AV-RUTLD,3,IRT,04/10/2021,16:00:00,RECOVR AUD,84404,175178
11516,R627,R063,00-03-01,SUTTER AV-RUTLD,3,IRT,04/12/2021,08:00:00,RECOVR AUD,84760,175856
11517,R627,R063,00-03-02,SUTTER AV-RUTLD,3,IRT,04/10/2021,16:00:00,RECOVR AUD,26291,82302


In [11]:
#Create a dataframe with dropped rows
turnstiles_df = turnstiles_df[turnstiles_df.DESC!= 'RECOVR AUD'] # Deleted Duplicates
turnstiles_df.shape

(2706153, 11)

## Counting entries for each unique turnstile

In [12]:
#As the entries and exits are cumulative. Utilizing .Shift() to calculate entries

In [13]:
#Combining columns C/A, UNIT and SCP to make a unique turnstile. STATION will remain as seperate column so analysis is easy later
turnstiles_df['Unique_Turnstile'] = turnstiles_df['C/A']+ turnstiles_df['UNIT'] + turnstiles_df['SCP']

In [14]:
pd = turnstiles_df

In [15]:
turnstiles_df.head() 

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Unique_Turnstile
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,03:00:00,REGULAR,7518158,2561766,A002R05102-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,07:00:00,REGULAR,7518162,2561773,A002R05102-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,11:00:00,REGULAR,7518183,2561806,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,15:00:00,REGULAR,7518267,2561847,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/16/2021,19:00:00,REGULAR,7518403,2561864,A002R05102-00-00


In [16]:
# Dropping unnecessary columns 
turnstiles_df_cleaned = turnstiles_df.drop(['C/A','UNIT','SCP','LINENAME','DIVISION','DESC'], axis=1)
turnstiles_df_cleaned.head()

,STATION,DATE,TIME,ENTRIES,EXITS,Unique_Turnstile
0,59 ST,01/16/2021,03:00:00,7518158,2561766,A002R05102-00-00
1,59 ST,01/16/2021,07:00:00,7518162,2561773,A002R05102-00-00
2,59 ST,01/16/2021,11:00:00,7518183,2561806,A002R05102-00-00
3,59 ST,01/16/2021,15:00:00,7518267,2561847,A002R05102-00-00
4,59 ST,01/16/2021,19:00:00,7518403,2561864,A002R05102-00-00


In [17]:
#In order to change the order of columns removing empty space
turnstiles_df_cleaned.columns = [column.strip() for column in turnstiles_df_cleaned.columns]
turnstiles_df_cleaned.columns

Index(['STATION', 'DATE', 'TIME', 'ENTRIES', 'EXITS', 'Unique_Turnstile'], dtype='object')

In [18]:
#Re-ordered columns 
turnstiles_df_cleaned = turnstiles_df_cleaned[['Unique_Turnstile', 'STATION', 'DATE', 'TIME', 'ENTRIES', 'EXITS']]
turnstiles_df_cleaned.head()

,Unique_Turnstile,STATION,DATE,TIME,ENTRIES,EXITS
0,A002R05102-00-00,59 ST,01/16/2021,03:00:00,7518158,2561766
1,A002R05102-00-00,59 ST,01/16/2021,07:00:00,7518162,2561773
2,A002R05102-00-00,59 ST,01/16/2021,11:00:00,7518183,2561806
3,A002R05102-00-00,59 ST,01/16/2021,15:00:00,7518267,2561847
4,A002R05102-00-00,59 ST,01/16/2021,19:00:00,7518403,2561864


In [19]:
#creating a mask to verify the understanding that each unique turstile each day records 6 entries
mask = turnstiles_df_cleaned[(turnstiles_df_cleaned["Unique_Turnstile"] == "A002R05102-00-00")&
                     (turnstiles_df_cleaned["STATION"] == "59 ST")&
                     (turnstiles_df_cleaned["DATE"] == "01/16/2021")]
mask.head()

,Unique_Turnstile,STATION,DATE,TIME,ENTRIES,EXITS
0,A002R05102-00-00,59 ST,01/16/2021,03:00:00,7518158,2561766
1,A002R05102-00-00,59 ST,01/16/2021,07:00:00,7518162,2561773
2,A002R05102-00-00,59 ST,01/16/2021,11:00:00,7518183,2561806
3,A002R05102-00-00,59 ST,01/16/2021,15:00:00,7518267,2561847
4,A002R05102-00-00,59 ST,01/16/2021,19:00:00,7518403,2561864


In [20]:
#As entries and exits are cumulative, using .shift() to create new columns 
turnstiles_df_cleaned[["PREV_ENTRIES", "PREV_EXITS", "PREV_DATE"]] = (turnstiles_df_cleaned[["ENTRIES", "EXITS", "DATE"]]
                                                       .apply(lambda grp: grp.shift(1)))

In [21]:
turnstiles_df_cleaned.head()

,Unique_Turnstile,STATION,DATE,TIME,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,PREV_DATE
0,A002R05102-00-00,59 ST,01/16/2021,03:00:00,7518158,2561766,NaN,NaN,NaN
1,A002R05102-00-00,59 ST,01/16/2021,07:00:00,7518162,2561773,7518158.0,2561766.0,01/16/2021
2,A002R05102-00-00,59 ST,01/16/2021,11:00:00,7518183,2561806,7518162.0,2561773.0,01/16/2021
3,A002R05102-00-00,59 ST,01/16/2021,15:00:00,7518267,2561847,7518183.0,2561806.0,01/16/2021
4,A002R05102-00-00,59 ST,01/16/2021,19:00:00,7518403,2561864,7518267.0,2561847.0,01/16/2021


In [22]:
#Deleting NaN row
turnstiles_df_cleaned.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [23]:
turnstiles_df_cleaned ['Entry_Count'] = turnstiles_df_cleaned['ENTRIES'] - turnstiles_df_cleaned['PREV_ENTRIES']
turnstiles_df_cleaned ['Exit_Count'] = turnstiles_df_cleaned['EXITS'] - turnstiles_df_cleaned['PREV_EXITS']

In [24]:
turnstiles_df_cleaned = turnstiles_df_cleaned.drop(['PREV_ENTRIES','PREV_EXITS', 'PREV_DATE','ENTRIES', 'EXITS' ], axis=1)
turnstiles_df_cleaned.head()

,Unique_Turnstile,STATION,DATE,TIME,Entry_Count,Exit_Count
1,A002R05102-00-00,59 ST,01/16/2021,07:00:00,4.0,7.0
2,A002R05102-00-00,59 ST,01/16/2021,11:00:00,21.0,33.0
3,A002R05102-00-00,59 ST,01/16/2021,15:00:00,84.0,41.0
4,A002R05102-00-00,59 ST,01/16/2021,19:00:00,136.0,17.0
5,A002R05102-00-00,59 ST,01/16/2021,23:00:00,87.0,8.0


In [25]:
turnstiles_df_cleaned.describe() # Looks like there are negative values and extremly large values

,Entry_Count,Exit_Count
count,2.706152e+06,2.706152e+06
mean,-2.776120e+00,-9.464431e-01
std,4.625890e+07,4.137238e+07
min,-2.144357e+09,-2.109011e+09
25%,1.000000e+00,3.000000e+00
50%,1.800000e+01,1.900000e+01
75%,6.400000e+01,6.300000e+01
max,2.139010e+09,2.122816e+09


In [39]:
mask = turnstiles_df_cleaned.loc[((turnstiles_df_cleaned['Entry_Count'] < 0) & (turnstiles_df_cleaned['Exit_Count'] < 0))]
mask.head()

,Unique_Turnstile,STATION,DATE,TIME,Entry_Count,Exit_Count,Date_Time


In [27]:
mask['STATION'].value_counts()

14 ST              1330
34 ST-PENN STA     1133
161/YANKEE STAD     769
GRD CNTRL-42 ST     723
50 ST               638
                   ... 
CORTELYOU RD         13
BEACH 105 ST         13
88 ST                13
DITMAS AV             4
138/GRAND CONC        1
Name: STATION, Length: 374, dtype: int64

In [28]:
turnstiles_df_cleaned['Entry_Count'] = turnstiles_df_cleaned.Entry_Count.abs()# converting values to positive using .abs()

In [29]:
turnstiles_df_cleaned['Exit_Count'] = turnstiles_df_cleaned.Exit_Count.abs()

In [30]:
turnstiles_df_cleaned.describe() 

,Entry_Count,Exit_Count
count,2.706152e+06,2.706152e+06
mean,1.764156e+06,1.452845e+06
std,4.622525e+07,4.134686e+07
min,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.000000e+00
50%,2.000000e+01,2.100000e+01
75%,6.900000e+01,6.800000e+01
max,2.144357e+09,2.122816e+09


In [31]:
turnstiles_df_cleaned.head()

,Unique_Turnstile,STATION,DATE,TIME,Entry_Count,Exit_Count
1,A002R05102-00-00,59 ST,01/16/2021,07:00:00,4.0,7.0
2,A002R05102-00-00,59 ST,01/16/2021,11:00:00,21.0,33.0
3,A002R05102-00-00,59 ST,01/16/2021,15:00:00,84.0,41.0
4,A002R05102-00-00,59 ST,01/16/2021,19:00:00,136.0,17.0
5,A002R05102-00-00,59 ST,01/16/2021,23:00:00,87.0,8.0


In [32]:
#Looks like the turnstiles are resetting and dropping them should be fine. 
#Average daily ridership in 2019 according to this website is around 260,000: https://www.timessquarenyc.org/do-business/market-research-data/transit-ridership
#will get rid of daily counts above 500,000.

In [33]:
def reset_counter(n):
    if n > 500000:
        return 0
    return n 

turnstiles_df_cleaned['Entry_Count'] = turnstiles_df_cleaned['Entry_Count'].apply(reset_counter)
turnstiles_df_cleaned['Exit_Count']= turnstiles_df_cleaned['Exit_Count'].apply(reset_counter)


In [34]:
turnstiles_df_cleaned.Entry_Count.max()

499827.0

In [35]:
#Exporting data to visualize in Tableau
turnstiles_df_cleaned.to_csv ('turnstiles_df_cleaned.csv')


In [36]:
%store turnstiles_df_cleaned

Stored 'turnstiles_df_cleaned' (DataFrame)
